<a href="https://colab.research.google.com/github/Nicole-Cheah/test1/blob/main/Nicole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 50.040 Natural Language Processing (Fall 2025) Homework 3

**Due 30/11/2025, 23:59 PM**


### STUDENT ID: 1007011

### Name: Nicole Cheah

### Students with whom you have discussed (if any):

## IMDB Sentiment Analysis

### Project Overview

This notebook implements a complete sentiment analysis pipeline for IMDB movie reviews using:
- **Model**: Qwen/Qwen3-0.6B (600M parameters)
- **Dataset**: stanfordnlp/imdb (50,000 movie reviews)
- **Task**: Binary sentiment classification (positive/negative)
- **Course**: 50.040 / NLP — IMDB Sentiment homework 3

### Methodology

We compare three approaches:
1. **Base Model**: Zero-shot performance without fine-tuning
2. **Full-Parameter SFT**: Fine-tuning all model parameters
3. **LoRA-SFT**: Parameter-efficient fine-tuning using LoRA (Low-Rank Adaptation)

Tips: If you don't have a GPU, we recommend using Kaggle & Colab resources to complete this experiment. Estimated GPU processing time is <2 hours.

Kaggle:  https://www.kaggle.com/

Colab: https://colab.research.google.com/


PS: It is important that all code runs completely, accounting for 10 points.

---
## Section 0: Environment Setup and Configuration

This section imports all necessary libraries, sets random seeds for reproducibility, and configures the training environment.

In [1]:
!pip install jsonlines

In [2]:
"""
Import required libraries and set environment variables
"""
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import json
import random
import numpy as np
import torch
import jsonlines
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    set_seed
)
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from typing import Dict, List, Tuple
import gc
from torch.utils.data import Dataset

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [3]:
"""
Set random seed for reproducibility across all frameworks
"""
RANDOM_SEED = 42
set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print(f"Random seed set to: {RANDOM_SEED}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Random seed set to: 42
PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB
CUDA memory: 39.56 GB


In [4]:
"""
GPU memory management utility

This function helps free up GPU memory between training runs
and provides visibility into current memory usage.
"""
def free_gpu(tag: str = ""):
    """Clean up GPU memory and print current usage."""
    gc.collect()
    if not torch.cuda.is_available():
        print(f"[free_gpu]{'['+tag+']' if tag else ''} CUDA not available.")
        return

    for d in range(torch.cuda.device_count()):
        try:
            torch.cuda.synchronize(d)
        except:
            pass
        with torch.cuda.device(d):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
        try:
            torch.cuda.synchronize(d)
        except:
            pass

    gc.collect()

    # Print memory usage
    stats = []
    for d in range(torch.cuda.device_count()):
        a = torch.cuda.memory_allocated(d) / 1048576
        r = torch.cuda.memory_reserved(d) / 1048576
        stats.append(f"GPU{d}: alloc={a:.1f}MB, reserved={r:.1f}MB")
    print(f"[free_gpu]{'['+tag+']' if tag else ''} " + " | ".join(stats))

free_gpu("initial")

[free_gpu][initial] GPU0: alloc=0.0MB, reserved=0.0MB


In [5]:
"""
Configuration - Optimized for 15GB GPU

These hyperparameters are carefully tuned to:
- Fit within 15GB GPU memory constraints
- Provide reasonable training time (~2-5 minutes per approach)
- Maintain model performance
"""
MODEL_NAME = "Qwen/Qwen3-0.6B"
MAX_SEQ_LENGTH = 512
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
WARMUP_STEPS = 20
MAX_TRAIN_SAMPLES = 2000
MAX_VALID_SAMPLES = 100
MAX_TEST_SAMPLES = 100
GRADIENT_ACCUMULATION_STEPS = 4


# Create directory structure
os.makedirs("data", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("figures", exist_ok=True)
os.makedirs("checkpoints/full_sft", exist_ok=True)
os.makedirs("checkpoints/lora_sft", exist_ok=True)

print("\n" + "="*60)
print("Configuration:")
print("="*60)
print(f"Model: {MODEL_NAME}")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Train batch size: {TRAIN_BATCH_SIZE}")
print(f"Eval batch size: {EVAL_BATCH_SIZE}")
print(f"Gradient accumulation steps: {GRADIENT_ACCUMULATION_STEPS}")
print(f"Effective batch size: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Number of epochs: {NUM_EPOCHS}")
print(f"Train samples: {MAX_TRAIN_SAMPLES}")
print(f"Validation samples: {MAX_VALID_SAMPLES}")
print(f"Test samples: {MAX_TEST_SAMPLES}")
print("="*60 + "\n")


Configuration:
Model: Qwen/Qwen3-0.6B
Max sequence length: 512
Train batch size: 8
Eval batch size: 8
Gradient accumulation steps: 4
Effective batch size: 32
Learning rate: 2e-05
Number of epochs: 1
Train samples: 2000
Validation samples: 100
Test samples: 100



---
## Section 1: Data Processing

In this section, we:
1. Load the IMDB dataset from HuggingFace
2. Convert reviews to instruction-following format
3. Split data into train, validation, and test sets
4. Save processed data in JSONL format

In [6]:
"""
Define data formatting functions

The instruction format helps the model understand the task:
Prompt: "Is this review positive or negative?\n\n{review}\n\nAnswer with only one word: positive or negative. Answer:"
Response: "positive" or "negative"
"""
def format_instruction(text: str, label: int) -> Dict[str, str]:
    """
    Convert IMDB sample to instruction format.

    Args:
        text: Original review text
        label: 0 (negative) or 1 (positive)

    Returns:
        Dictionary with prompt and response
    """
    #
    sentiment = "positive" if label == 1 else "negative"
    return {
        "prompt": f"Is this review positive or negative?\n\n{text}\n\nAnswer with only one word: positive or negative. Answer:",
        "response": sentiment,
        "label": label
    }

def save_dataset(data, output_path: str):
    """Save dataset to JSONL format."""
    with jsonlines.open(output_path, mode='w') as writer:
        for item in data:
            formatted = format_instruction(item['text'], item['label'])
            writer.write(formatted)
    print(f"Saved {len(data)} samples to {output_path}")

print("✅ Data formatting functions defined")

✅ Data formatting functions defined




#### Q1.1: Why do we need to reformat the dataset instead of using the original one directly? [written] (4 point)





We need to reformat the dataset due to the following reasons:

1. LLMs are trained to respond to instructions
2. The format helps the model to understand what task to perform
3. This provides context for the expected output format
4. Lastly, it enables the model to generalize to similar instruction based tasks


#### Q1.2: How can we train LLM if we don’t perform this conversion?  [written] (4 point)




Without the conversion, there are two other ways to train this LLM:

1. Use next-token prediction on the raw text, but this method  does not teach task specific behaviour
2. Finetune the model on text with implicit sentiment signals, although the performance could be worse
3. Use traditional classification heads (but this would result in a loss of the generative capability of the model). The model would not learn to follow instructions or respond in the desired format.

In [7]:
"""
Load and split IMDB dataset

The IMDB dataset contains 50,000 movie reviews:
- 25,000 for training
- 25,000 for testing

We subsample to reduce training time while maintaining balanced classes.
"""
print("Loading IMDB dataset...")
dataset = load_dataset("stanfordnlp/imdb")

print(f"Original train samples: {len(dataset['train'])}")
print(f"Original test samples: {len(dataset['test'])}")

# Split train into train and validation    (7 point)
### YOUR CODE HERE
from sklearn.model_selection import train_test_split
# Get all training data
all_train = dataset['train']

train_indices = list(range(len(all_train)))
random.shuffle(train_indices)
sampled_indices = train_indices[:MAX_TRAIN_SAMPLES + MAX_VALID_SAMPLES]

### END OF YOUR CODE

# Sample training data & random selection
### YOUR CODE HERE
sampled_data = all_train.select(sampled_indices)

### END OF YOUR CODE

# Split into train and validation
### YOUR CODE HERE
train_size = MAX_TRAIN_SAMPLES
valid_size = MAX_VALID_SAMPLES

train_subset = sampled_data.select(range(train_size))
valid_subset = sampled_data.select(range(train_size, train_size + valid_size))
### END OF YOUR CODE

# Limit test data
### YOUR CODE HERE
test_indices = list(range(len(dataset['test'])))
random.shuffle(test_indices)
test_subset = dataset['test'].select(test_indices[:MAX_TEST_SAMPLES])

### END OF YOUR CODE

print(f"\nDataset splits:")
print(f"Train: {len(train_subset)} samples")
print(f"Validation: {len(valid_subset)} samples")
print(f"Test: {len(test_subset)} samples")

# Save to JSONL
save_dataset(train_subset, "data/train.jsonl")
save_dataset(valid_subset, "data/valid.jsonl")
save_dataset(test_subset, "data/test.jsonl")

print("\n✅ Data processing complete")

Loading IMDB dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Original train samples: 25000
Original test samples: 25000

Dataset splits:
Train: 2000 samples
Validation: 100 samples
Test: 100 samples
Saved 2000 samples to data/train.jsonl
Saved 100 samples to data/valid.jsonl
Saved 100 samples to data/test.jsonl

✅ Data processing complete


---
## Section 2: Tokenizer Setup

Load the tokenizer for the Qwen model and configure special tokens for padding.

In [8]:
"""
Load and configure tokenizer

The tokenizer converts text to token IDs that the model understands.
We set the pad_token to eos_token if not already defined.
"""
print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set pad token if not exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token to eos_token")

print(f"\nTokenizer vocab size: {len(tokenizer)}")
print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print("\n✅ Tokenizer loaded successfully")

Loading tokenizer from Qwen/Qwen3-0.6B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]


Tokenizer vocab size: 151669
PAD token: <|endoftext|> (ID: 151643)
EOS token: <|im_end|> (ID: 151645)

✅ Tokenizer loaded successfully


---
## Section 3: Dataset Class

Define a custom PyTorch Dataset class for instruction-following format with:
- Proper tokenization of prompts and responses
- Label masking (only train on response tokens, not prompt)
- Padding and truncation to max length

In [9]:
"""
Helper function to load JSONL files
"""
def load_jsonl(file_path: str) -> List[Dict]:
    """Load JSONL file."""
    data = []
    with jsonlines.open(file_path) as reader:
        for obj in reader:
            data.append(obj)
    return data

print("✅ JSONL loader defined")

✅ JSONL loader defined


In [10]:
"""
Define custom Dataset class for instruction-following

Key features:
1. Tokenizes prompt and response separately
2. Sets labels to -100 for prompt tokens (ignored in loss)
3. Sets labels to actual tokens for response (trained)
4. Handles padding and truncation
"""
class InstructionDataset(Dataset):
    def __init__(self, file_path: str, tokenizer, max_length: int):
        self.data = load_jsonl(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = item['prompt']
        response = item['response']

        # Tokenize prompt and response separately

        prompt_encoded = self.tokenizer(
            prompt,
            add_special_tokens=True,
            truncation=False
        )

        response_text = f" {response}{self.tokenizer.eos_token}"
        response_encoded = self.tokenizer(
            response_text,
            add_special_tokens=False,
            truncation=False
        )

        # Combine input_ids  & outputs
        # (2 point)
        ### YOUR CODE HERE
        input_ids = prompt_encoded['input_ids'] + response_encoded['input_ids']

        # Create labels: -100 for prompt (ignored), actual tokens for response
        # (2 point)
         ### YOUR CODE HERE
        labels = [-100] * len(prompt_encoded['input_ids']) + response_encoded['input_ids']

        # Truncate
        # (3 point)
        ### YOUR CODE HERE
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length]
            labels = labels[:self.max_length]


        # Pad to max_length
        # (3 point)
        ### YOUR CODE HERE
        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
            labels = labels + [-100] * padding_length
            attention_mask = [1] * (self.max_length - padding_length) + [0] * padding_length
        else:
            attention_mask = [1] * self.max_length

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

print("✅ InstructionDataset class defined")

✅ InstructionDataset class defined


In [11]:
"""
Create dataset instances for train, validation, and test
"""
train_dataset = InstructionDataset("data/train.jsonl", tokenizer, MAX_SEQ_LENGTH)
valid_dataset = InstructionDataset("data/valid.jsonl", tokenizer, MAX_SEQ_LENGTH)
test_dataset = InstructionDataset("data/test.jsonl", tokenizer, MAX_SEQ_LENGTH)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Valid dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Verify dataset
print("\n" + "="*60)
print("TOKENIZED DATASET SAMPLE INSPECTION")
print("="*60)
sample = train_dataset[0]
print(f"Dataset sample shapes:")
print(f"  Input IDs shape: {sample['input_ids'].shape}")
print(f"  Labels shape: {sample['labels'].shape}")
print(f"  Attention mask shape: {sample['attention_mask'].shape}")

print(f"\nInput IDs (first 20 tokens): {sample['input_ids'][:20].tolist()}")
print(f"Labels (first 20 tokens): {sample['labels'][:20].tolist()}")
print(f"Attention mask (first 20 tokens): {sample['attention_mask'][:20].tolist()}")

# Decode to verify
decoded_input = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)
print(f"\nDecoded input (first 200 chars): {decoded_input[:200]}...")

# Show where labels start being actual tokens (not -100)
label_start = None
for i, label in enumerate(sample['labels']):
    if label != -100:
        label_start = i
        break
print(f"\nLabels analysis:")
print(f"  Prompt tokens (label=-100): {label_start}")
print(f"  Response tokens (label=actual): {(sample['labels'] != -100).sum().item()}")
print(f"  Padding tokens (label=-100): {((sample['labels'] == -100) & (sample['attention_mask'] == 0)).sum().item()}")

# Decode only the response part
if label_start is not None:
    response_tokens = sample['input_ids'][label_start:]
    response_tokens = response_tokens[response_tokens != tokenizer.pad_token_id]
    decoded_response = tokenizer.decode(response_tokens, skip_special_tokens=True)
    print(f"  Decoded response: '{decoded_response}'")

print("="*60 + "\n")
print("✅ Datasets created successfully")

Train dataset size: 2000
Valid dataset size: 100
Test dataset size: 100

TOKENIZED DATASET SAMPLE INSPECTION
Dataset sample shapes:
  Input IDs shape: torch.Size([512])
  Labels shape: torch.Size([512])
  Attention mask shape: torch.Size([512])

Input IDs (first 20 tokens): [3872, 419, 3395, 6785, 476, 8225, 1939, 49454, 2435, 391, 777, 11, 7607, 23845, 323, 67294, 28392, 312, 19892, 632]
Labels (first 20 tokens): [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Attention mask (first 20 tokens): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Decoded input (first 200 chars): Is this review positive or negative?

Eric Clapton, Jack Bruce and Ginger Baker re-unite to play all their songs from 35 years ago when they formed a trio called "Cream." Those were the psychedelic da...

Labels analysis:
  Prompt tokens (label=-100): 323
  Response tokens (label=actual): 2
  Padding tokens (label=-100): 187
  Dec

---
## Section 4: Evaluation Functions

Define functions for:
- Model inference (sentiment prediction)
- Comprehensive evaluation with metrics
- Confusion matrix generation

In [12]:
"""
Prediction function for sentiment inference

This function:
1. Takes a prompt as input
2. Generates model predictions (greedy decoding)
3. Extracts sentiment from generated text
"""
def predict_sentiment(model, tokenizer, prompt: str, max_new_tokens: int = 10) -> str:
    """
    Generate sentiment prediction for a given prompt.

    Args:
        model: Trained model
        tokenizer: Tokenizer
        prompt: Input prompt
        max_new_tokens: Maximum tokens to generate

    Returns:
        Predicted sentiment ('positive' or 'negative')
    """
    model.eval()

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    # Remove token_type_ids if exists
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode only the generated part
    # (2 point)
    ### YOUR CODE HERE
    input_length = inputs['input_ids'].shape[1]
    generated_ids = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
     ### END OF YOUR CODE

    # Extract sentiment
    # (2 point)
    ### YOUR CODE HERE
    generated_text_lower = generated_text.lower().strip()
    if 'positive' in generated_text_lower:
        return 'positive'
    elif 'negative' in generated_text_lower:
        return 'negative'
    else:
        # Default to negative if unclear
        return 'negative'
    ### END OF YOUR CODE

print("✅ predict_sentiment function defined")

✅ predict_sentiment function defined


In [13]:
"""
Comprehensive evaluation function

This function:
1. Evaluates model on entire test set
2. Calculates multiple metrics (accuracy, F1, precision, recall)
3. Generates confusion matrix
4. Saves failed cases for analysis
"""
def evaluate_model(model, tokenizer, test_file: str, output_prefix: str) -> Dict:
    """
    Evaluate model on test set.

    Args:
        model: Trained model
        tokenizer: Tokenizer
        test_file: Path to test JSONL file
        output_prefix: Prefix for output files (e.g., 'base', 'full', 'lora')

    Returns:
        Dictionary with evaluation metrics
    """
    print(f"\nEvaluating {output_prefix} model...")

    test_data = load_jsonl(test_file)

    y_true = []
    y_pred = []
    failed_cases = []

    for item in tqdm(test_data, desc="Evaluating"):
        prompt = item['prompt']
        true_label = item['response']

        pred_label = predict_sentiment(model, tokenizer, prompt)

        y_true.append(true_label)
        y_pred.append(pred_label)

        # Collect failed cases
        if pred_label != true_label:
            failed_cases.append({
                "prompt": prompt[:200],
                "true_label": true_label,
                "predicted_label": pred_label
            })

    # Calculate metrics
    # ACC F1 Precision Recall (4 point)
    ### YOUR CODE HERE
    accuracy = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    macro_precision = precision_score(y_true, y_pred, average='macro')
    macro_recall = recall_score(y_true, y_pred, average='macro')
    ### END OF YOUR CODE

    metrics = {
        "accuracy": float(accuracy),
        "macro_f1": float(macro_f1),
        "macro_precision": float(macro_precision),
        "macro_recall": float(macro_recall),
        "total_samples": len(y_true),
        "correct_predictions": int(sum([1 for t, p in zip(y_true, y_pred) if t == p]))
    }

    # Save metrics
    with open(f"logs/metrics_{output_prefix}.json", "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"\n{output_prefix.upper()} Model Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Macro F1: {macro_f1:.4f}")
    print(f"Macro Precision: {macro_precision:.4f}")
    print(f"Macro Recall: {macro_recall:.4f}")

    # Save failed cases (first 5)
    if failed_cases:
        with open(f"logs/failed_cases_{output_prefix}.json", "w") as f:
            json.dump(failed_cases[:5], f, indent=2, ensure_ascii=False)
        print(f"\nFailed cases saved to logs/failed_cases_{output_prefix}.json")

    # Generate confusion matrix & show png
    # (2 point)
    ### YOUR CODE HERE
    cm = confusion_matrix(y_true, y_pred, labels=['negative', 'positive'])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['negative', 'positive'],
                yticklabels=['negative', 'positive'])
    plt.title(f'Confusion Matrix - {output_prefix.upper()}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'figures/confmat_{output_prefix}.png', dpi=300, bbox_inches='tight')
    plt.close()
    ### END OF YOUR CODE

    print(f"Confusion matrix saved to figures/confmat_{output_prefix}.png")

    return metrics

print("✅ evaluate_model function defined")

✅ evaluate_model function defined



## Section 5: Base Model Testing (Zero-Shot)

Evaluate the pre-trained Qwen model without any fine-tuning to establish a baseline performance.

In [14]:
"""
Load base model for zero-shot evaluation

This establishes our baseline - how well does the model perform
on sentiment analysis without any task-specific training?
"""
print("\n" + "="*60)
print("Base Model Testing (No Fine-tuning)")
print("="*60 + "\n")

free_gpu("before base model")

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else None,
    trust_remote_code=True,
    device_map="auto"
)

base_model.resize_token_embeddings(len(tokenizer))
base_model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Base model loaded successfully")


Base Model Testing (No Fine-tuning)

[free_gpu][before base model] GPU0: alloc=0.0MB, reserved=0.0MB


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Base model loaded successfully


In [15]:
"""
Evaluate base model on test set
"""
# using before define evaluate_model function
# your code here
metrics_base = evaluate_model(base_model, tokenizer, "data/test.jsonl", "base")


# Clean up to free memory for next stage
del base_model
free_gpu("after base model")


Evaluating base model...


Evaluating: 100%|██████████| 100/100 [00:43<00:00,  2.32it/s]



BASE Model Metrics:
Accuracy: 0.8000
Macro F1: 0.7960
Macro Precision: 0.8190
Macro Recall: 0.7975

Failed cases saved to logs/failed_cases_base.json
Confusion matrix saved to figures/confmat_base.png
[free_gpu][after base model] GPU0: alloc=8.1MB, reserved=1136.0MB


---
## Section 6: Full-Parameter SFT [code] (10 point)

Fine-tune ALL model parameters using supervised fine-tuning (SFT).

**Approach**: Updates all 600M parameters of the Qwen model

In [16]:
"""
Load model for full-parameter fine-tuning
"""
print("\n" + "="*60)
print("Full-Parameter SFT")
print("="*60 + "\n")

free_gpu("before full SFT")

model_full = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else None,
    trust_remote_code=True,
    device_map="auto"
)

model_full.resize_token_embeddings(len(tokenizer))
model_full.config.pad_token_id = tokenizer.pad_token_id
model_full.config.use_cache = False

if hasattr(model_full, "gradient_checkpointing_enable"):
    model_full.gradient_checkpointing_enable()
    print("✅ Gradient checkpointing enabled (saves memory)")

# Count parameters
# (2 point)
### YOUR CODE HERE
total_params = sum(p.numel() for p in model_full.parameters())
trainable_params = sum(p.numel() for p in model_full.parameters() if p.requires_grad)
### END OF YOUR CODE

print(f"\nModel loaded:")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")


Full-Parameter SFT

[free_gpu][before full SFT] GPU0: alloc=8.1MB, reserved=1136.0MB
✅ Gradient checkpointing enabled (saves memory)

Model loaded:
Total parameters: 595,776,512
Trainable parameters: 595,776,512 (100.00%)


In [17]:
"""
Configure training arguments and initialize trainer

Key settings:
- bf16: Use bfloat16 for faster training and lower memory
- gradient_accumulation_steps: Simulate larger batch size
- save_strategy: Save checkpoint after each epoch
"""
# 6 point
training_args_full = TrainingArguments(
    ### YOUR CODE HERE
    output_dir="checkpoints/full_sft",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    bf16=True,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    logging_dir="logs/full_sft",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    report_to="none",
    seed=RANDOM_SEED,
    dataloader_num_workers=2,
    dataloader_pin_memory=False,
    max_grad_norm=1.0,
    optim="adamw_torch",
    ### END OF YOUR CODE
)

print("✅ Training arguments configured for full SFT")

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer
)

print("✅ Trainer initialized for full SFT")

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Training arguments configured for full SFT
✅ Trainer initialized for full SFT


In [18]:
"""
Train the full-parameter model

This will take approximately 2-3 minutes on a 15GB GPU.
"""
print("\n🚀 Starting Full-Parameter SFT Training...\n")

start_time = time.time()
train_result = trainer_full.train()
training_time_full = time.time() - start_time

print(f"\n✅ Training completed in {training_time_full:.2f} seconds ({training_time_full/60:.2f} minutes)")

# Save model
trainer_full.save_model("checkpoints/full_sft/final_model")
tokenizer.save_pretrained("checkpoints/full_sft/final_model")
print("✅ Model saved to checkpoints/full_sft/final_model")

# Save training metrics
train_metrics_full = {
    "training_time_seconds": training_time_full,
    "training_time_minutes": training_time_full / 60,
    "train_loss": train_result.training_loss,
    "total_params": total_params,
    "trainable_params": trainable_params,
    "trainable_params_percentage": 100 * trainable_params / total_params
}

with open("logs/full_sft_train_metrics.json", "w") as f:
    json.dump(train_metrics_full, f, indent=2)

print("✅ Training metrics saved to logs/full_sft_train_metrics.json")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



🚀 Starting Full-Parameter SFT Training...



Epoch,Training Loss,Validation Loss
1,0.132400,0.077601



✅ Training completed in 98.31 seconds (1.64 minutes)
✅ Model saved to checkpoints/full_sft/final_model
✅ Training metrics saved to logs/full_sft_train_metrics.json


In [19]:
"""
Evaluate full SFT model on test set
"""


metrics_full = evaluate_model(model_full, tokenizer, "data/test.jsonl", "full")


# Clean up to free memory for LoRA training
del model_full, trainer_full
free_gpu("after full SFT")


Evaluating full model...


Evaluating: 100%|██████████| 100/100 [00:11<00:00,  8.91it/s]



FULL Model Metrics:
Accuracy: 0.8800
Macro F1: 0.8800
Macro Precision: 0.8804
Macro Recall: 0.8804

Failed cases saved to logs/failed_cases_full.json
Confusion matrix saved to figures/confmat_full.png
[free_gpu][after full SFT] GPU0: alloc=16.2MB, reserved=1168.0MB


---
## Section 7: LoRA-SFT (Parameter-Efficient Fine-Tuning)

Apply LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning.

**Approach**: Only updates low-rank adapter matrices (~0.5% of parameters)
**Training time**: ~1-2 minutes (faster than full SFT)
**Memory usage**: ~8-10 GB (lower than full SFT)
**Performance**: Often comparable to full fine-tuning

In [20]:
"""
Load model and apply LoRA configuration

LoRA parameters:
- r=8: Rank of low-rank matrices (controls adapter size)
- lora_alpha=16: Scaling factor for LoRA weights
- target_modules: Which attention layers to adapt (q, k, v, o projections)
- lora_dropout=0.05: Dropout for regularization
"""
print("\n" + "="*60)
print("LoRA-SFT (Parameter-Efficient Fine-Tuning)")
print("="*60 + "\n")

free_gpu("before LoRA SFT")

model_lora = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

model_lora.resize_token_embeddings(len(tokenizer))
model_lora.config.pad_token_id = tokenizer.pad_token_id
model_lora.config.use_cache = False
if hasattr(model_lora, "gradient_checkpointing_enable"):
    model_lora.gradient_checkpointing_enable()
    print("Gradient checkpointing enabled for LoRA")
print("✅ Base model loaded for LoRA")

# Apply LoRA
# r=8, lora_alpha=16, target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
# (4 point)
lora_config = LoraConfig(
    ### YOUR CODE HERE
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
    ### END OF YOUR CODE
)

model_lora = get_peft_model(model_lora, lora_config)

# Print trainable parameters
model_lora.print_trainable_parameters()

# Count parameters
total_params_lora = sum(p.numel() for p in model_lora.parameters())
trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

print(f"\nLoRA model:")
print(f"Total parameters: {total_params_lora:,}")
print(f"Trainable parameters: {trainable_params_lora:,} ({100 * trainable_params_lora / total_params_lora:.4f}%)")
print(f"\n📊 Parameter reduction: {100 * (1 - trainable_params_lora / trainable_params):.2f}%")


LoRA-SFT (Parameter-Efficient Fine-Tuning)

[free_gpu][before LoRA SFT] GPU0: alloc=16.2MB, reserved=1168.0MB
Gradient checkpointing enabled for LoRA
✅ Base model loaded for LoRA
trainable params: 2,293,760 || all params: 598,070,272 || trainable%: 0.3835

LoRA model:
Total parameters: 598,070,272
Trainable parameters: 2,293,760 (0.3835%)

📊 Parameter reduction: 99.61%


#### Q7.1: Please examine the model architecture of Qwen-3-0.6B (https://huggingface.co/Qwen/Qwen3-0.6B) and mathematically derive the number of trainable parameters under the current configuration [write] (10 points)


For Qwen-3-0.6B:
- hidden_size (d) = 896
- num_layers (L) = 28
- r (LoRA rank) = 8
- target_modules = 4 (q_proj, k_proj, v_proj, o_proj)

For each target module per layer:
- Matrix A: d × r = 896 × 8 = 7,168 parameters
- Matrix B: r × d = 8 × 896 = 7,168 parameters
- Total per module: 7,168 + 7,168 = 14,336 parameters

Per layer (4 modules):
- 14,336 × 4 = 57,344 parameters

Total trainable (28 layers):
- 57,344 × 28 = 1,605,632 parameters ≈ 1.61M parameters

This number is approximately 0.27% of the base model's parameters (600M).

In [21]:
"""
Configure training arguments and initialize trainer for LoRA
"""
training_args_lora = TrainingArguments(
    output_dir="checkpoints/lora_sft",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    bf16=True,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    logging_dir="logs/lora_sft",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    report_to="none",
    seed=RANDOM_SEED,
    dataloader_num_workers=2,
    dataloader_pin_memory=False,
    max_grad_norm=1.0,
    optim="adamw_torch",
)

print("✅ Training arguments configured for LoRA SFT")

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer
)

print("✅ Trainer initialized for LoRA SFT")

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Training arguments configured for LoRA SFT
✅ Trainer initialized for LoRA SFT


In [22]:
"""
Train the LoRA model

This should be faster than full SFT due to fewer trainable parameters.
"""
print("\n🚀 Starting LoRA-SFT Training...\n")

start_time = time.time()
train_result_lora = trainer_lora.train()
training_time_lora = time.time() - start_time

print(f"\n✅ Training completed in {training_time_lora:.2f} seconds ({training_time_lora/60:.2f} minutes)")

# Save LoRA adapter
model_lora.save_pretrained("checkpoints/lora_sft/final_adapter")
tokenizer.save_pretrained("checkpoints/lora_sft/final_adapter")
print("✅ LoRA adapter saved to checkpoints/lora_sft/final_adapter")

# Save training metrics
train_metrics_lora = {
    "training_time_seconds": training_time_lora,
    "training_time_minutes": training_time_lora / 60,
    "train_loss": train_result_lora.training_loss,
    "total_params": total_params_lora,
    "trainable_params": trainable_params_lora,
    "trainable_params_percentage": 100 * trainable_params_lora / total_params_lora
}

with open("logs/lora_sft_train_metrics.json", "w") as f:
    json.dump(train_metrics_lora, f, indent=2)

print("✅ Training metrics saved to logs/lora_sft_train_metrics.json")

# Print speedup
speedup = training_time_full / training_time_lora
print(f"\n⚡ LoRA training speedup: {speedup:.2f}x faster than full SFT")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



🚀 Starting LoRA-SFT Training...



Epoch,Training Loss,Validation Loss
1,4.496000,3.695383


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(



✅ Training completed in 93.47 seconds (1.56 minutes)


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✅ LoRA adapter saved to checkpoints/lora_sft/final_adapter
✅ Training metrics saved to logs/lora_sft_train_metrics.json

⚡ LoRA training speedup: 1.05x faster than full SFT


In [23]:
"""
Merge LoRA weights with base model and evaluate

Merging is optional but makes deployment easier (single model file).
"""
print("\nMerging LoRA weights with base model...")
# merge (2 point)
### YOUR CODE HERE
model_lora_merged = model_lora.merge_and_unload()
### END OF YOUR CODE
model_lora_merged.save_pretrained("checkpoints/lora_sft/merged_model")
tokenizer.save_pretrained("checkpoints/lora_sft/merged_model")
print("✅ Merged model saved to checkpoints/lora_sft/merged_model")

# Evaluate LoRA SFT model
metrics_lora = evaluate_model(model_lora_merged, tokenizer, "data/test.jsonl", "lora")

# Clean up
del model_lora, model_lora_merged, trainer_lora
free_gpu("after LoRA SFT")


Merging LoRA weights with base model...
✅ Merged model saved to checkpoints/lora_sft/merged_model

Evaluating lora model...


Evaluating: 100%|██████████| 100/100 [00:41<00:00,  2.43it/s]



LORA Model Metrics:
Accuracy: 0.7700
Macro F1: 0.7647
Macro Precision: 0.8044
Macro Recall: 0.7733

Failed cases saved to logs/failed_cases_lora.json
Confusion matrix saved to figures/confmat_lora.png
[free_gpu][after LoRA SFT] GPU0: alloc=16.2MB, reserved=1168.0MB



## Section 8: Performance Comparison and Visualization [code]

Compare all three approaches (Base, Full-SFT, LoRA-SFT) across multiple dimensions:
- Accuracy and F1 scores
- Training time
- Number of trainable parameters

In [24]:
"""
Print comparison table
"""
print("\n" + "="*60)
print("Performance Comparison")
print("="*60 + "\n")

models = ['Base Model', 'Full-SFT', 'LoRA-SFT']
accuracies = [
    metrics_base['accuracy'],
    metrics_full['accuracy'],
    metrics_lora['accuracy']
]
f1_scores = [
    metrics_base['macro_f1'],
    metrics_full['macro_f1'],
    metrics_lora['macro_f1']
]

print(f"{'Model':<20} {'Accuracy':<15} {'Macro F1':<15}")
print("-" * 50)
for i, model in enumerate(models):
    print(f"{model:<20} {accuracies[i]:<15.4f} {f1_scores[i]:<15.4f}")
print("\n")


Performance Comparison

Model                Accuracy        Macro F1       
--------------------------------------------------
Base Model           0.8000          0.7960         
Full-SFT             0.8800          0.8800         
LoRA-SFT             0.7700          0.7647         




In [25]:
"""
Create comparison bar charts
"""
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# Accuracy comparison
bars1 = ax1.bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 1.0)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

# F1 Score comparison
bars2 = ax2.bar(models, f1_scores, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Macro F1 Score', fontsize=12, fontweight='bold')
ax2.set_title('Model F1 Score Comparison', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 1.0)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Comparison chart saved to figures/model_comparison.png")

✅ Comparison chart saved to figures/model_comparison.png


In [26]:
"""
Create comprehensive comparison with all metrics
"""
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_names = ['Accuracy', 'Macro F1', 'Precision', 'Recall']
metrics_data = [
    [metrics_base['accuracy'], metrics_full['accuracy'], metrics_lora['accuracy']],
    [metrics_base['macro_f1'], metrics_full['macro_f1'], metrics_lora['macro_f1']],
    [metrics_base['macro_precision'], metrics_full['macro_precision'], metrics_lora['macro_precision']],
    [metrics_base['macro_recall'], metrics_full['macro_recall'], metrics_lora['macro_recall']]
]

for idx, (ax, metric_name, metric_values) in enumerate(zip(axes.flat, metrics_names, metrics_data)):
    bars = ax.bar(models, metric_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel(metric_name, fontsize=11, fontweight='bold')
    ax.set_title(f'{metric_name} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.0)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_xticklabels(models, rotation=15, ha='right')

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.suptitle('Comprehensive Model Performance Comparison', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()
plt.savefig('figures/comprehensive_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Comprehensive comparison chart saved to figures/comprehensive_comparison.png")

/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')


✅ Comprehensive comparison chart saved to figures/comprehensive_comparison.png


In [27]:
"""
Generate comprehensive project summary
"""
summary = {
    "project_info": {
        "model": MODEL_NAME,
        "dataset": "stanfordnlp/imdb",
        "task": "Binary sentiment classification",
        "random_seed": RANDOM_SEED
    },
    "configuration": {
        "max_seq_length": MAX_SEQ_LENGTH,
        "train_batch_size": TRAIN_BATCH_SIZE,
        "eval_batch_size": EVAL_BATCH_SIZE,
        "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
        "effective_batch_size": TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS,
        "learning_rate": LEARNING_RATE,
        "num_epochs": NUM_EPOCHS
    },
    "data_splits": {
        "train": len(train_dataset),
        "validation": len(valid_dataset),
        "test": len(test_dataset)
    },
    "base_model": {
        "test_metrics": metrics_base
    },
    "full_sft": {
        "training": train_metrics_full,
        "test_metrics": metrics_full
    },
    "lora_sft": {
        "training": train_metrics_lora,
        "test_metrics": metrics_lora
    },
    "comparison": {
        "accuracy": {
            "base": metrics_base['accuracy'],
            "full_sft": metrics_full['accuracy'],
            "lora_sft": metrics_lora['accuracy']
        },
        "training_time_minutes": {
            "full_sft": train_metrics_full['training_time_minutes'],
            "lora_sft": train_metrics_lora['training_time_minutes']
        },
        "trainable_params_millions": {
            "full_sft": train_metrics_full['trainable_params'] / 1e6,
            "lora_sft": train_metrics_lora['trainable_params'] / 1e6
        }
    }
}

with open("logs/project_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*60)
print("PROJECT SUMMARY")
print("="*60)
print(json.dumps(summary, indent=2))
print("\n✅ Project summary saved to logs/project_summary.json")


PROJECT SUMMARY
{
  "project_info": {
    "model": "Qwen/Qwen3-0.6B",
    "dataset": "stanfordnlp/imdb",
    "task": "Binary sentiment classification",
    "random_seed": 42
  },
  "configuration": {
    "max_seq_length": 512,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "gradient_accumulation_steps": 4,
    "effective_batch_size": 32,
    "learning_rate": 2e-05,
    "num_epochs": 1
  },
  "data_splits": {
    "train": 2000,
    "validation": 100,
    "test": 100
  },
  "base_model": {
    "test_metrics": {
      "accuracy": 0.8,
      "macro_f1": 0.7960016319869441,
      "macro_precision": 0.818961818961819,
      "macro_recall": 0.7975190076030412,
      "total_samples": 100,
      "correct_predictions": 80
    }
  },
  "full_sft": {
    "training": {
      "training_time_seconds": 98.30704808235168,
      "training_time_minutes": 1.638450801372528,
      "train_loss": 1.7452164500478715,
      "total_params": 595776512,
      "trainable_params": 595776512,
      "train

In [28]:
"""
Print final comparison table
"""
print("\n" + "="*80)
print("FINAL COMPARISON TABLE")
print("="*80)
print(f"{'Metric':<30} {'Base Model':<20} {'Full-SFT':<20} {'LoRA-SFT':<20}")
print("-"*80)
print(f"{'Test Accuracy':<30} {metrics_base['accuracy']:>19.4f} {metrics_full['accuracy']:>19.4f} {metrics_lora['accuracy']:>19.4f}")
print(f"{'Test Macro F1':<30} {metrics_base['macro_f1']:>19.4f} {metrics_full['macro_f1']:>19.4f} {metrics_lora['macro_f1']:>19.4f}")
print(f"{'Test Precision':<30} {metrics_base['macro_precision']:>19.4f} {metrics_full['macro_precision']:>19.4f} {metrics_lora['macro_precision']:>19.4f}")
print(f"{'Test Recall':<30} {metrics_base['macro_recall']:>19.4f} {metrics_full['macro_recall']:>19.4f} {metrics_lora['macro_recall']:>19.4f}")
print("-"*80)
print(f"{'Training Time (min)':<30} {'N/A':>19} {train_metrics_full['training_time_minutes']:>19.2f} {train_metrics_lora['training_time_minutes']:>19.2f}")
print(f"{'Trainable Params (M)':<30} {'N/A':>19} {train_metrics_full['trainable_params']/1e6:>19.2f} {train_metrics_lora['trainable_params']/1e6:>19.2f}")
print(f"{'Trainable Params %':<30} {'N/A':>19} {train_metrics_full['trainable_params_percentage']:>19.2f} {train_metrics_lora['trainable_params_percentage']:>19.2f}")
print("="*80)

print("\n✅ All tasks completed successfully!")
print("\n📁 Generated files:")
print("  - Data: data/train.jsonl, data/valid.jsonl, data/test.jsonl")
print("  - Models: checkpoints/full_sft/, checkpoints/lora_sft/")
print("  - Metrics: logs/metrics_*.json")
print("  - Figures: figures/*.png")
print("  - Summary: logs/project_summary.json")


FINAL COMPARISON TABLE
Metric                         Base Model           Full-SFT             LoRA-SFT            
--------------------------------------------------------------------------------
Test Accuracy                               0.8000              0.8800              0.7700
Test Macro F1                               0.7960              0.8800              0.7647
Test Precision                              0.8190              0.8804              0.8044
Test Recall                                 0.7975              0.8804              0.7733
--------------------------------------------------------------------------------
Training Time (min)                            N/A                1.64                1.56
Trainable Params (M)                           N/A              595.78                2.29
Trainable Params %                             N/A              100.00                0.38

✅ All tasks completed successfully!

📁 Generated files:
  - Data: data/train.jsonl

The LoRA model underperformed compared to the base model in this experiment (77% vs 80%). This could be attributed to:

Limited training data (2000 samples with 1 epoch)
Conservative hyperparameters (r=8 may be too low for this task)
Insufficient training time (single epoch may not allow LoRA adapters to converge)


#### Q8.1: Why is LoRA-SFT training time shorter? [written] (5 points)



Firstly, LoRA has fewer parameters to update, it only updates around 1.6M parameters, compared to the original 600M. Since LoRA has fewer parameters to update in Adam/AdamW, the optimizer steps are much faster.

Next, LoRA uses reduced gradient computation, as the backpropogation is only done through low rank matrices.

LoRA also uses lower memory bandwidth as there is lesser data transfer between the GPU memory.

Lastly, LoRA has better GPU utilization as the compute is more focused on forward pass.

### Other Question

For the robustness and error analysis experiments, I used the Full-SFT model (88% accuracy) rather than LoRA-SFT (77% accuracy), as it demonstrated superior performance on the test set. This allows for more reliable analysis of model behavior in challenging scenarios.

#### Q9: For example, if we switch to a different prompting format, will accuracy drop significantly when the train and test prompts are not completely consistent? Please demonstrate with experiments. (10 points)

In [42]:
def test_prompt_robustness_with_model(model, tokenizer):
    """Test model with different prompt formats"""

    # Original format
    original_prompt = "Is this review positive or negative?\n\n{text}\n\nAnswer with only one word: positive or negative. Answer:"

    # Alternative formats
    alt_formats = [
        "Review: {text}\n\nSentiment:",
        "Classify the sentiment: {text}\n\nAnswer:",
        "{text}\n\nIs this positive or negative?",
        "Determine if positive or negative: {text}",
    ]

    # Test on sample data
    test_samples = load_jsonl("data/test.jsonl")[:20]

    results = {}
    print("\n" + "="*60)
    print("PROMPT ROBUSTNESS TEST")
    print("="*60)

    for format_name, format_str in [("original", original_prompt)] + [(f"alt_{i}", f) for i, f in enumerate(alt_formats)]:
        correct = 0
        for item in test_samples:
            # Extract review text from original prompt
            review_text = item['prompt'].split('\n\n')[1].split('\n\n')[0]
            new_prompt = format_str.format(text=review_text)

            pred = predict_sentiment(model, tokenizer, new_prompt)
            if pred == item['response']:
                correct += 1

        accuracy = correct / len(test_samples)
        results[format_name] = accuracy
        print(f"{format_name}: Accuracy = {accuracy:.4f} ({correct}/{len(test_samples)})")

    return results



In [43]:
# Load the Full-SFT model for testing
model_for_testing = AutoModelForCausalLM.from_pretrained(
    "checkpoints/full_sft/final_model",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True,
    device_map="auto"
)

# Run robustness test with Full-SFT
prompt_robustness_results = test_prompt_robustness_with_model(model_for_testing, tokenizer)




PROMPT ROBUSTNESS TEST
original: Accuracy = 0.8500 (17/20)
alt_0: Accuracy = 0.6500 (13/20)
alt_1: Accuracy = 0.7500 (15/20)
alt_2: Accuracy = 0.6500 (13/20)
alt_3: Accuracy = 0.6000 (12/20)



#### Q10: How can we increase robustness in this scenario, given that users’ queries are highly diverse? (5 points)


Here are 5 strategies to increase robustness:

1. Data Augmentation: Train with multiple prompt variations
2. Prompt Ensembling: Use multiple prompts at inference and aggregate results
3. Instruction Tuning: Train on diverse instruction formats
4. Soft Prompts: Learn continuous prompt embeddings
5. Multi-task Learning: Train on related tasks simultaneously

#### Q11: Please propose five different test cases that will cause the SFT-trained model to classify incorrectly, and verify them. (5 points)

I designed five test cases:

**Case 1: Sarcasm**
- Text: "Oh wonderful, another two hours of my life wasted on this masterpiece."
- True: Negative | Predicted: Positive
- **Why it failed**: The model focuses on the positive words ("wonderful", "masterpiece") and is unable to detect sarcastic intent. Sarcasm requires contextual reasoning
  beyond lexical analysis.

**Case 2: Backhanded Compliment**
- Text: "Good job, I suppose if the intention was to create the most forgettable
  movie in the last decade."
- True: Negative | Predicted: Positive
- **Why it failed**: The phrase "Good job" at the beginning triggers positive
  classification despite the negative continuation. The model lacks
  long-range dependency understanding and misses the negative part.

**Case 3: Implicit Negative Comparison**
- Text: "I've seen better performances at the local high school theater."
- True: Negative | Predicted: Positive
- **Why it failed**: The criticism is implicit rather than explicit. The model
  does not understand that comparing professional work unfavorably to amateur
  performances is an insult.

**Case 4: Positive Lexical Cue with Negative Meaning**
- Text: "My expectations were low, but wow, they found a way to go lower."
- True: Negative | Predicted: Positive
- **Why it failed**: The exclamation "wow" is typically positive, but here it
  expresses dissappointment. The model relies too heavily on sentiment
  lexicons rather than contextual meaning.

**Case 5: Context Dependent Reference**
- Text: "This movie makes Radhe look like The Godfather in comparison."
- True: Negative | Predicted: Positive
- **Why it failed**: Understanding this statement requires some contextual knowledge. The Godfather is rated as the 2nd best movie on IMDb, with a rating of 9.2 and 2.2M reviews, while Radhe ranks 8 on Lowest Rated movies list, with a rating of 1.8 and 181K reviews. This statement implies that the movie is so bad that it makes a bad film look good in comparison.

**Key Insights:**
- The model performs well on simple and direct sentiment (88% accuracy overall)
- It fails on language requiring pragmatic understanding
- Fine-tuning on 2000 simple IMDB reviews didn't expose the model to complex
  linguistic phenomena like sarcasm, irony, and implicit comparisons

In [87]:
challenging_cases = [
    {
        "text": "Oh wonderful, another two hours of my life wasted on this masterpiece.",
        "true_label": "negative",
        "challenge": "Sarcasm"
    },
    {
        "text": "Good job, I suppose if the intention was to create the most forgettable movie in the last decade.",
        "true_label": "negative",
        "challenge": "Backhanded compliment"
    },
    {
        "text": "I've seen better performances at the local high school theater.",
        "true_label": "negative",
        "challenge": "Implicit negative comparison"
    },
    {
        "text": "My expectations were low, but wow, they found a way to go lower.",
        "true_label": "negative",
        "challenge": "Positive Lexical Cue (wow) With Negative Meaning"
    },
    {
        "text": "This movie makes Radhe look like The Godfather in comparison.",
        "true_label": "negative",
        "challenge": "Context Dependent Reference"
    }
]

def test_challenging_cases(model, tokenizer):
    """Test model on challenging cases"""
    print("CHALLENGING TEST CASES")

    for i, case in enumerate(challenging_cases, 1):
        prompt = f"Is this review positive or negative?\n\n{case['text']}\n\nAnswer with only one word: positive or negative. Answer:"
        pred = predict_sentiment(model, tokenizer, prompt)

        print(f"\nCase {i}: {case['challenge']}")
        print(f"Text: {case['text']}")
        print(f"True label: {case['true_label']}")
        print(f"Predicted: {pred}")
        print(f"Result: {'✓ CORRECT' if pred == case['true_label'] else '✗ WRONG'}")

In [88]:
# Run error analysis with Full-SFT
test_challenging_cases(model_for_testing, tokenizer)

CHALLENGING TEST CASES

Case 1: Sarcasm
Text: Oh wonderful, another two hours of my life wasted on this masterpiece.
True label: negative
Predicted: positive
Result: ✗ WRONG

Case 2: Backhanded compliment
Text: Good job, I suppose if the intention was to create the most forgettable movie in the last decade.
True label: negative
Predicted: positive
Result: ✗ WRONG

Case 3: Implicit negative comparison
Text: I've seen better performances at the local high school theater.
True label: negative
Predicted: positive
Result: ✗ WRONG

Case 4: Positive Lexical Cue (wow) With Negative Meaning
Text: My expectations were low, but wow, they found a way to go lower.
True label: negative
Predicted: positive
Result: ✗ WRONG

Case 5: Context Dependent Reference
Text: This movie makes Radhe look like The Godfather in comparison.
True label: negative
Predicted: positive
Result: ✗ WRONG
